In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, concatenate, GlobalAveragePooling2D, Dense, Dropout

# Define the input shape
input_shape = (None, None, 3)  # Change this according to your data

# Define the input layer
input_layer = Input(shape=input_shape)

# Apply 1x16 convolutional layer stride 1 with depth of 64
conv1 = Conv2D(filters=64, kernel_size=(1, 16), strides=1, padding='same')(input_layer)

# Apply 3x3 convolutional layer stride 1
conv2 = Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')(conv1)

# Apply this output to 4 different layers parallely of size 1x3, 3x1, 1x5, 5x1
conv3_1 = Conv2D(filters=64, kernel_size=(1, 3), strides=1, padding='same')(conv2)
conv3_2 = Conv2D(filters=64, kernel_size=(3, 1), strides=1, padding='same')(conv2)
conv3_3 = Conv2D(filters=64, kernel_size=(1, 5), strides=1, padding='same')(conv2)
conv3_4 = Conv2D(filters=64, kernel_size=(5, 1), strides=1, padding='same')(conv2)

# Add all four outputs together
merged_conv3 = concatenate([conv3_1, conv3_2, conv3_3, conv3_4], axis=-1)

# Apply max-pool then batch normalization then relu
max_pool = MaxPooling2D(pool_size=(2, 2), strides=2)(merged_conv3)
batch_norm = BatchNormalization()(max_pool)
relu_activation = Activation('relu')(batch_norm)

# Apply 5x5 convolutional layer stride 2
conv5x5 = Conv2D(filters=64, kernel_size=(5, 5), strides=2, padding='same')(relu_activation)

# Depthwise convolution
depthwise_conv = Conv2D(filters=64, kernel_size=(1, 1), strides=1, padding='same')(conv5x5)

# Apply 1x7 convolution depth 64 and add it with the previous saved output
conv1x7 = Conv2D(filters=64, kernel_size=(1, 7), strides=1, padding='same')(depthwise_conv)
merged_output = concatenate([conv1x7, merged_conv3], axis=-1)

# Apply 7x1 convolutional layer stride 1 depth 64
conv7x1 = Conv2D(filters=64, kernel_size=(7, 1), strides=1, padding='same')(merged_output)

# Batch normalization
batch_norm_2 = BatchNormalization()(conv7x1)

# Softmax layer
softmax_layer = Activation('softmax')(batch_norm_2)

# Save this output to one variable
output_1 = softmax_layer

# Apply 5x1 convolutional layer stride 1 then 3x1 convolutional layer stride 1
conv5x1_2 = Conv2D(filters=64, kernel_size=(5, 1), strides=1, padding='same')(batch_norm_2)
conv3x1 = Conv2D(filters=64, kernel_size=(3, 1), strides=1, padding='same')(conv5x1_2)

# Save this output to another variable
output_2 = conv3x1

# Apply 5x1 convolutional layer stride 1 then 3x1 convolutional layer stride 1
conv5x1_3 = Conv2D(filters=64, kernel_size=(5, 1), strides=1, padding='same')(conv3x1)
conv3x1_2 = Conv2D(filters=64, kernel_size=(3, 1), strides=1, padding='same')(conv5x1_3)

# Flatten
flatten_layer = Flatten()(conv3x1_2)

# Apply global average pooling on both outputs
global_avg_pool_1 = GlobalAveragePooling2D()(output_1)
global_avg_pool_2 = GlobalAveragePooling2D()(output_2)

# Concatenate the global average pooling results
concatenated_global_avg = concatenate([global_avg_pool_1, global_avg_pool_2], axis=-1)

# Fully connected layer 1
fc1 = Dense(128)(concatenated_global_avg)
dropout1 = Dropout(0.2)(fc1)

# Fully connected layer 2
fc2 = Dense(64)(dropout1)
dropout2 = Dropout(0.2)(fc2)

# Fully connected layer 3
fc3 = Dense(10)(dropout2)  # Change this according to your number of classes

# Define the output layer
output_layer = Activation('softmax')(fc3)

# Create the model
custom_model = Model(inputs=input_layer, outputs=output_layer)

# Print the model summary
custom_model.summary()
custom_model.save("September_2.h5")

: 